In [1]:
#Kết nối với drive cá nhân
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 35.6 MB/s 
     |████████████████████████████████| 86 kB 4.5 MB/s 
     |████████████████████████████████| 596 kB 67.6 MB/s 
     |████████████████████████████████| 6.6 MB 55.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install sentencepiece

In [ ]:
!pip install vncorenlp

In [ ]:
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

In [ ]:
import os
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from transformers import TFAutoModel, AutoTokenizer
import transformers
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, BertTokenizer, BertForSequenceClassification
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, recall_score

In [ ]:
from vncorenlp import VnCoreNLP
vncorenlp = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split 

In [ ]:
import pandas as pd
data = pd.read_excel("/content/gdrive/MyDrive/Luan_van/data_crawl_19.3.xlsx", index_col=False)

In [ ]:
data_Sentiment = data['Sentiment']
data_Sentiment = data_Sentiment.astype(int)
data['Sentiment'] = data_Sentiment
data

In [ ]:
train, test= train_test_split(data, test_size=0.3, random_state=42)

In [ ]:
X_train = train['Text']
y_train = train['Sentiment'].values
X_test = test['Text']
y_test = test['Sentiment'].values

In [ ]:
#pre-process
import re
import numpy as np

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def preprocess(text, tokenized=True, lowercased=True):
    # text = ViTokenizer.tokenize(text)
    # text = ' '.join(vncorenlp.tokenize(text)[0])
    #text = filter_stop_words(text, stopwords)
    text = deEmojify(text)
    text = text.lower() if lowercased else text
    if tokenized:
        pre_text = ""
        sentences = vncorenlp.tokenize(text)
        for sentence in sentences:
            pre_text += " ".join(sentence)
        text = pre_text
    return text

def pre_process_features(X, y, tokenized=True, lowercased=True):
    X = [preprocess(str(p), tokenized=tokenized, lowercased=lowercased) for p in list(X)]
    for idx, ele in enumerate(X):
        if not ele:
            np.delete(X, idx)
            np.delete(y, idx)
    return X, y
def preprocess_phobert(text, tokenized=True, lowercased=True):
    # text = ViTokenizer.tokenize(text)
    # text = ' '.join(vncorenlp.tokenize(text)[0])
    #text = filter_stop_words(text, stopwords)
    text = deEmojify(text)
    text = text.lower() if lowercased else text
    if tokenized:
        pre_text = ""
        sentences = vncorenlp.tokenize(text)
        for sentence in sentences:
            pre_text += " ".join(sentence)
        text = pre_text
    return text

In [ ]:
max_length = 256

def norm(x):
    if len(x[0]) < 200:
        temp = torch.cat((x[0], torch.tensor([1]*(200-len(x[0])))), 0)
    else:
        temp = torch.tensor([x[0][:200].tolist()])
    return temp

In [ ]:
import torch
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, recall_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, BertTokenizer, BertForSequenceClassification
from sklearn.metrics import classification_report
model = AutoModelForSequenceClassification.from_pretrained("/content/gdrive/MyDrive/Luan_van/phobert", num_labels = 3)
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base",use_fast=False)

In [ ]:
#dư đoán nhãn
import torch.nn.functional as F
import time

texts = "Không nên tiêm vaccine cho người lớn"
texts = preprocess_phobert(texts)


tokenized_texts = tokenizer.encode_plus(texts, add_special_tokens = True, max_length=200,truncation = True, padding = "max_length", return_attention_mask = True, return_tensors = "pt")
print(texts)
start = time.time()
y_pred_classify2 = model(tokenized_texts['input_ids'])
y_pred = torch.argmax(y_pred_classify2.logits)
end  = time.time()
t = end - start
print(t)
if y_pred == 2:
  print("positive" )
if y_pred == 1:
  print("neutral" )
if y_pred == 0:
  print("negative" )

In [ ]:
tokenized_texts['input_ids']

In [ ]:
print(tokenized_texts['input_ids'].shape)

In [ ]:
batch_size = 4
x = torch.tensor([[1]*200]*batch_size)

In [ ]:
torch_out = model(x)

In [ ]:
torch.onnx.export(model,              
                  x,
                  "model.onnx",  
                  export_params=True,       
                  opset_version=11,         
                  do_constant_folding=True,  
                  input_names = ['input'],   
                  output_names = ['output'], 
                  dynamic_axes={'input' : {0 : '-1'},    
                                'output' : {0 : '-1'}})

In [ ]:
!pip install onnx #Open Neural Network Exchange
import onnx

onnx_model = onnx.load("model.onnx")
onnx.checker.check_model(onnx_model)

In [ ]:
!pip install onnxruntime

In [ ]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("model.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
import time
start = time.time()
# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(tokenized_texts['input_ids'])}
ort_outs = ort_session.run(None, ort_inputs)
end = time.time()
print(end - start)
# compare ONNX Runtime and PyTorch results
print("Exported model has been") 